In [4]:
import sys
import pandas as pd
import boto3
from time import time
import subprocess
sys.path.append('../../src')
import cb_utils
import datetime
import json

pd.options.display.max_columns = 500
con = cb_utils.get_engine()

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('/Users/bp/Downloads/member_doc_file_router_external_files.csv')
df.head()

,id,original_name,s3_bucket,s3_key,hash,is_already_transferred,is_successful_download,is_successful_upload,is_successful_job_enqueued,is_still_being_written,has_error,error,job_id,path,file_size,modified_at,retrieval_batch_id,ftp_server_id,inserted_at,updated_at,is_ignored
0,98788,cb_ip_auth_20211005.txt,cb-analytics-us-east-2-prd,inbound/uhc_tn/authorization_notifications/163...,cb_ip_auth_20211005.txt-2021-10-5-10-13-5-37906,False,True,True,True,False,False,NaN,190754,NaN,37906,2021-10-05 10:13:05,104546,5,2021-10-05 11:10:04.306068,2021-10-05 11:10:04.421982,NaN
1,98789,cb_op_auth_20211005.txt,cb-analytics-us-east-2-prd,inbound/uhc_tn/authorization_notifications/163...,cb_op_auth_20211005.txt-2021-10-5-10-13-5-20079,False,True,True,True,False,False,NaN,190755,NaN,20079,2021-10-05 10:13:05,104546,5,2021-10-05 11:10:04.424588,2021-10-05 11:10:04.507572,NaN
2,99263,cb_ip_auth_20211006.txt,cb-analytics-us-east-2-prd,inbound/uhc_tn/authorization_notifications/163...,cb_ip_auth_20211006.txt-2021-10-6-10-9-51-43627,False,True,True,True,False,False,NaN,192051,NaN,43627,2021-10-06 10:09:51,105683,5,2021-10-06 11:10:06.990966,2021-10-06 11:10:07.156720,NaN
3,99264,cb_op_auth_20211006.txt,cb-analytics-us-east-2-prd,inbound/uhc_tn/authorization_notifications/163...,cb_op_auth_20211006.txt-2021-10-6-10-9-51-34478,False,True,True,True,False,False,NaN,192052,NaN,34478,2021-10-06 10:09:51,105683,5,2021-10-06 11:10:07.159226,2021-10-06 11:10:07.230596,NaN
4,99496,Authenticare10-05-2021.xlsx,cb-analytics-us-east-2-prd,inbound/uhc_ks/authorization/1633540201_99496_...,Authenticare10-05-2021.xlsx-2021-10-5-18-1-42-...,False,True,True,True,False,False,NaN,192544,NaN,4798952,2021-10-05 18:01:42,105999,176,2021-10-06 17:10:01.264796,2021-10-06 17:10:02.289531,NaN


In [24]:
files = []
for idx, r in df.iterrows():
    files.append((r.id, r.original_name, r.s3_bucket, format_path(r.s3_key)))
#     print(r)

In [23]:
import os
import subprocess
def format_path(p):
    path_parts = p.split("/")
    path_parts.insert(3, "processed")
    return "/".join(path_parts)

In [38]:
results = []
for i, f, b, p in files:
    try:
        if '.xlsx' in f:
            print(f)
            s3.download_file(b, p, f)

            d = pd.read_excel(f, sheet_name='Sheet1')
#             num_lines = sum(1 for line in open(f))
            num_lines = d.shape[0]
            results.append({'file_id': i, 'num_lines': num_lines})
            os.remove(f)
    except:
        print('failed for: ', i, f)

Authenticare10-05-2021.xlsx
Authenticare10-06-2021.xlsx
Authenticare10-07-2021.xlsx
Authenticare10-08-2021.xlsx
Authenticare10-11-2021.xlsx
Authenticare10-12-2021.xlsx
Authenticare10-13-2021.xlsx
Authenticare10-14-2021.xlsx
Authenticare10-15-2021.xlsx
Authenticare10-18-2021.xlsx
Authenticare10-19-2021.xlsx
TN_2021Q4_XWALK_2021_10_18.xlsx
failed for:  103100 TN_2021Q4_XWALK_2021_10_18.xlsx
Authenticare10-20-2021.xlsx
Authenticare10-21-2021.xlsx
Authenticare10-22-2021.xlsx
Authenticare10-25-2021.xlsx


In [39]:
rdf = pd.DataFrame(results)
rdf.head()

,file_id,num_lines
0,99496,108923
1,99498,109372
2,99926,109642
3,100276,110099
4,101102,110535


In [40]:
eng = cb_utils.get_engine()

rdf.to_sql('line_counts_20211026', eng, schema='junk', if_exists='append', method='multi', index=False)

In [ ]:
# 
failed for:  103101 TN_Expanded_Member_Eligibility_20211018.txt --
failed for:  104421 TN_Member_Eligibility_20211018.txt -- 8937
failed for:  104420 TN_Member_Eligibility_20210816.txt -- 6220

In [ ]:
AWS_ACCESS_KEY_ID
The access key for your AWS account.

AWS_SECRET_ACCESS_KEY
The secret key for your AWS account.

In [5]:
bucket_name = 'cb-member-doc-us-east-2-prd'
prefix = 'parsed_claim_file_archive/'

In [6]:
client = boto3.client('s3')

In [7]:
response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

In [7]:
contents = response['Contents']

In [52]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)    
files = []
for file in bucket.objects.filter(Prefix=prefix):
    #compare dates 
    if (file.last_modified).replace(tzinfo = None).date() >= datetime.datetime(2021,1, 5,tzinfo = None).date():
        files.append(file)

In [53]:
len(files)

551

In [54]:
files = [f for f in files if '999.x12.json' not in f.key]

In [55]:
len(files)

365

In [56]:
files = [f for f in files if '.277Ack.x12.json' not in f.key]

In [57]:
len(files)

189

In [58]:
files = [f for f in files if '.277.json' in f.key]

In [59]:
len(files)

35

In [60]:
hs_files = [
'10472_UB920166_2021010406020901.277.json',
'10652_UB920166_2021010501521901.277.json',
'10653_UB920166_2021010502035101.277.json',
'10654_UB920166_2021010502071001.277.json',
'10655_UB920166_2021010502194201.277.json',
'10656_UB920166_2021010502411601.277.json',
'10657_UB920166_2021010504160001.277.json',
'10658_UB920166_2021010504160002.277.json',
'10659_UB920166_2021010504160003.277.json',
'10660_UB920166_2021010504160004.277.json',
'10661_UB920166_2021010504160005.277.json',
'10662_UB920166_2021010504160006.277.json',
'10663_UB920166_2021010504160101.277.json',
]
def hs_file(f):
    is_hs_file = False
    for hs_f in hs_files:
        if hs_f in f.key:
            is_hs_file = True
            break
    return is_hs_file

In [61]:
files = [f for f in files if not hs_file(f)]

In [62]:
len(files)

23

In [36]:
for f in files[:23]:
    print(f.key, f.last_modified)

parsed_claim_file_archive/10474_GATE0166_2021010413064501.277.json 2021-01-05 00:00:22+00:00
parsed_claim_file_archive/10475_GATE0166_2021010416160401.277.json 2021-01-05 00:00:23+00:00
parsed_claim_file_archive/10476_GATE0166_2021010420005201.277.json 2021-01-05 06:00:23+00:00
parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json 2021-01-05 18:00:25+00:00
parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json 2021-01-05 18:00:25+00:00
parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json 2021-01-05 18:00:26+00:00
parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json 2021-01-06 00:00:24+00:00
parsed_claim_file_archive/10740_GATE0166_2021010512312601.277.json 2021-01-06 00:00:25+00:00
parsed_claim_file_archive/10741_GATE0166_2021010516160501.277.json 2021-01-06 00:00:25+00:00
parsed_claim_file_archive/10742_UB920166_2021010516020001.277.json 2021-01-06 00:00:26+00:00
parsed_claim_file_archive/10743_GATE0166_2021010520080901.277.json 202

In [37]:
potential = [
'parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json',
'parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json',
'parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json',
]

In [38]:
for f in files:
    if f.key in potential:
        print(f)

s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json')
s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json')
s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json')


In [70]:
content = [(f, f.get()['Body'].read().decode('utf-8')) for f in files]

In [71]:
# json.loads?
# c = content[0]['Body'].read().decode('utf-8')

In [74]:
for f, c in content:
    if '100000035' in c:
        if '3785' in c:
            print('one', f)
            file_content_3785 = json.loads(c)
        if '3778' in c:
            print('two', f)
            file_content_3778 = json.loads(c)

one s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json')
two s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json')


In [75]:
file_content_3778

{'document': [{'billing_provider_name': 'CAREBRIDGE MEDICAL GROUP, PC',
   'billing_provider_npi': '1124656616',
   'check_date': None,
   'check_number': '',
   'claim_amount': '0',
   'claim_external_status_code': 'A6:21',
   'claim_external_status_description': 'Missing or invalid information. Note: At least one other status code is required to identify the missing or invalid information.',
   'claim_status': 'U',
   'clearinghouse_id': None,
   'date_of_service': '2020-10-13',
   'date_of_services': '20201013-20201013',
   'is_claim_accepted': False,
   'is_claim_denied': False,
   'is_claim_paid': False,
   'is_claim_rejected': True,
   'is_submitter_accepted': True,
   'paid_amount': '',
   'patient_control_num': '3778',
   'patient_fname': 'SHIRLEY',
   'patient_id': '106538691',
   'patient_lname': 'LINCOLN',
   'payer_clrhs_name': 'INGENIX - EDIS',
   'payer_entity_identifier_code': 'AY',
   'payer_entity_type_qualifier': '2',
   'payer_identifier': '9080180109',
   'process_d